## Dataset selection for further analysis - Model Building

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load our data: october 2019 bicing

In [2]:
estat = pd.read_csv('../DATASETS/2019_10_Octubre_BicingNou_ESTACIONS.csv')
estat.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_updated,ttl
0,1,25,24,1,4,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30
1,2,12,11,1,15,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30
2,3,19,19,0,8,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30
3,4,15,15,0,3,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30
4,5,35,35,0,3,1,1,1,1.569881e+09,True,IN_SERVICE,1569880820,30


#### Convert timestamp into datetime:

In [3]:
from datetime import datetime

estat['last_reported_2'] = pd.to_datetime(estat['last_reported'], unit='s')
estat['last_updated_2'] = pd.to_datetime(estat['last_updated'], unit='s')


In [4]:
informacio = pd.read_csv('../DATASETS/2019_10_Octubre_BicingNou_INFORMACIO.csv')
informacio.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,cross_street,last_updated,ttl,nearby_distance
0,1,"C/ GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397952,2.180042,21.0,"C/ GRAN VIA CORTS CATALANES, 760",8908,30,NaN,1569880813,22,NaN
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395530,2.177060,21.0,"C/ ROGER DE FLOR, 126",8908,27,NaN,1569880813,22,NaN
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394055,2.181299,22.0,"C/ NÀPOLS, 82",8908,27,NaN,1569880813,22,NaN
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393480,2.181555,21.0,"C/ RIBES, 13",8908,21,NaN,1569880813,22,NaN
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391075,2.180223,16.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8908,39,NaN,1569880813,22,NaN


In [5]:
informacio['last_updated_2'] = pd.to_datetime(informacio['last_updated'], unit='s')

In [6]:
pd.set_option('display.max_rows', 500)
informacio.groupby('station_id').capacity.unique().head(400)

station_id
1                             [30]
2                             [27]
3                             [27]
4                             [21]
5                             [39]
6                             [39]
7                             [27]
8                             [27]
9                             [27]
11                            [21]
12                            [26]
13                            [54]
14                        [27, 26]
15                            [21]
17                            [42]
18                            [27]
19                            [30]
20                            [18]
21                            [21]
22                            [19]
23                            [27]
24                            [30]
25                            [21]
26                            [18]
27                            [21]
28                            [23]
29                            [23]
30                            [21]
31       

### Cleaning

In [7]:
df_mapa = informacio.drop_duplicates(subset=['station_id'], keep='first')
df_mapa['station_id'] = df_mapa['station_id'].astype(str)
df_mapa

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,cross_street,last_updated,ttl,nearby_distance,last_updated_2
0,1,"C/ GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397952,2.180042,21.0,"C/ GRAN VIA CORTS CATALANES, 760",8908,30,NaN,1569880813,22,NaN,2019-09-30 22:00:13
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395530,2.177060,21.0,"C/ ROGER DE FLOR, 126",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394055,2.181299,22.0,"C/ NÀPOLS, 82",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393480,2.181555,21.0,"C/ RIBES, 13",8908,21,NaN,1569880813,22,NaN,2019-09-30 22:00:13
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391075,2.180223,16.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8908,39,NaN,1569880813,22,NaN,2019-09-30 22:00:13
5,6,"PG. LLUIS COMPANYS, 18 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391349,2.180610,16.0,"PG. LLUIS COMPANYS, 18 (ARC TRIOMF)",8908,39,NaN,1569880813,22,NaN,2019-09-30 22:00:13
6,7,"PG. PUJADES, 1 (JUTJATS)",ELECTRICBIKESTATION,41.388856,2.183251,16.0,"PG. PUJADES, 1 (JUTJATS)",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
7,8,"PG. PUJADES, 2",ELECTRICBIKESTATION,41.389088,2.183568,11.0,"PG. PUJADES, 2",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
8,9,"AV. MARQUÉS DE L'ARGENTERA,13",ELECTRICBIKESTATION,41.384532,2.184921,11.0,"AV. MARQUÉS DE L'ARGENTERA,13",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
9,11,"PG. MARITIM, 11 (DAVANT PL. BRUGADA)",ELECTRICBIKESTATION,41.381689,2.193914,1.0,"PG. MARITIM, 11 (DAVANT PL. BRUGADA)",8908,21,NaN,1569880813,22,NaN,2019-09-30 22:00:13


In [8]:
import folium
from folium.plugins import MarkerCluster

In [9]:
from geopy.geocoders import Nominatim

In [10]:
geolocator=Nominatim(timeout=3)

In [11]:
informacio.physical_configuration.value_counts()

ELECTRICBIKESTATION    3618734
Name: physical_configuration, dtype: int64

In [12]:
info_reduced = informacio.drop_duplicates(subset=['station_id', 'capacity'], keep='first')
info_reduced

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,cross_street,last_updated,ttl,nearby_distance,last_updated_2
0,1,"C/ GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397952,2.180042,21.0,"C/ GRAN VIA CORTS CATALANES, 760",8908,30,NaN,1569880813,22,NaN,2019-09-30 22:00:13
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395530,2.177060,21.0,"C/ ROGER DE FLOR, 126",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394055,2.181299,22.0,"C/ NÀPOLS, 82",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393480,2.181555,21.0,"C/ RIBES, 13",8908,21,NaN,1569880813,22,NaN,2019-09-30 22:00:13
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391075,2.180223,16.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8908,39,NaN,1569880813,22,NaN,2019-09-30 22:00:13
5,6,"PG. LLUIS COMPANYS, 18 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391349,2.180610,16.0,"PG. LLUIS COMPANYS, 18 (ARC TRIOMF)",8908,39,NaN,1569880813,22,NaN,2019-09-30 22:00:13
6,7,"PG. PUJADES, 1 (JUTJATS)",ELECTRICBIKESTATION,41.388856,2.183251,16.0,"PG. PUJADES, 1 (JUTJATS)",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
7,8,"PG. PUJADES, 2",ELECTRICBIKESTATION,41.389088,2.183568,11.0,"PG. PUJADES, 2",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
8,9,"AV. MARQUÉS DE L'ARGENTERA,13",ELECTRICBIKESTATION,41.384532,2.184921,11.0,"AV. MARQUÉS DE L'ARGENTERA,13",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
9,11,"PG. MARITIM, 11 (DAVANT PL. BRUGADA)",ELECTRICBIKESTATION,41.381689,2.193914,1.0,"PG. MARITIM, 11 (DAVANT PL. BRUGADA)",8908,21,NaN,1569880813,22,NaN,2019-09-30 22:00:13


In [13]:
l = info_reduced.station_id.value_counts()
l

373    7
46     5
349    4
381    3
112    3
269    3
34     3
380    3
273    2
234    2
41     2
292    2
129    2
382    2
93     2
168    2
33     2
45     2
368    2
369    2
226    2
50     2
240    2
329    2
153    2
60     2
352    2
351    2
332    2
250    2
66     2
76     2
260    2
170    2
166    2
14     2
410    2
111    2
428    2
204    2
297    2
299    2
418    2
106    2
224    2
104    2
284    2
144    2
401    2
223    2
395    2
312    2
125    2
314    2
393    2
115    1
140    1
118    1
141    1
116    1
139    1
142    1
127    1
117    1
143    1
138    1
130    1
126    1
123    1
137    1
136    1
135    1
119    1
134    1
133    1
132    1
124    1
131    1
120    1
121    1
122    1
128    1
496    1
145    1
174    1
176    1
177    1
178    1
179    1
180    1
182    1
183    1
184    1
185    1
186    1
187    1
188    1
189    1
190    1
191    1
192    1
193    1
194    1
195    1
175    1
173    1
146    1
171    1
147    1
148    1
149    1
1

In [14]:
info_reduced_2 = info_reduced.drop_duplicates(subset=['station_id'], keep='first')
info_reduced_2

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,cross_street,last_updated,ttl,nearby_distance,last_updated_2
0,1,"C/ GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397952,2.180042,21.0,"C/ GRAN VIA CORTS CATALANES, 760",8908,30,NaN,1569880813,22,NaN,2019-09-30 22:00:13
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395530,2.177060,21.0,"C/ ROGER DE FLOR, 126",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394055,2.181299,22.0,"C/ NÀPOLS, 82",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393480,2.181555,21.0,"C/ RIBES, 13",8908,21,NaN,1569880813,22,NaN,2019-09-30 22:00:13
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391075,2.180223,16.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8908,39,NaN,1569880813,22,NaN,2019-09-30 22:00:13
5,6,"PG. LLUIS COMPANYS, 18 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391349,2.180610,16.0,"PG. LLUIS COMPANYS, 18 (ARC TRIOMF)",8908,39,NaN,1569880813,22,NaN,2019-09-30 22:00:13
6,7,"PG. PUJADES, 1 (JUTJATS)",ELECTRICBIKESTATION,41.388856,2.183251,16.0,"PG. PUJADES, 1 (JUTJATS)",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
7,8,"PG. PUJADES, 2",ELECTRICBIKESTATION,41.389088,2.183568,11.0,"PG. PUJADES, 2",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
8,9,"AV. MARQUÉS DE L'ARGENTERA,13",ELECTRICBIKESTATION,41.384532,2.184921,11.0,"AV. MARQUÉS DE L'ARGENTERA,13",8908,27,NaN,1569880813,22,NaN,2019-09-30 22:00:13
9,11,"PG. MARITIM, 11 (DAVANT PL. BRUGADA)",ELECTRICBIKESTATION,41.381689,2.193914,1.0,"PG. MARITIM, 11 (DAVANT PL. BRUGADA)",8908,21,NaN,1569880813,22,NaN,2019-09-30 22:00:13


In [15]:
list_cap = []
for i in info_reduced.station_id:
    if info_reduced.station_id.value_counts()[i] > 1 and i not in list_cap:
        list_cap.append(i)

In [16]:
list_cap

[14,
 33,
 34,
 41,
 45,
 46,
 50,
 60,
 66,
 76,
 93,
 104,
 106,
 111,
 112,
 125,
 129,
 144,
 153,
 166,
 168,
 170,
 204,
 223,
 224,
 226,
 234,
 240,
 250,
 260,
 269,
 273,
 284,
 292,
 297,
 299,
 312,
 314,
 329,
 332,
 349,
 351,
 352,
 368,
 369,
 373,
 380,
 381,
 382,
 393,
 395,
 401,
 410,
 418,
 428]

In [17]:
len(list_cap)

55

In [18]:
info_reduced_3 = info_reduced_2.drop(['address', 'cross_street', 'last_updated', 'ttl', 'nearby_distance', 'last_updated_2'], 1)
info_reduced_3

,station_id,name,physical_configuration,lat,lon,altitude,post_code,capacity
0,1,"C/ GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397952,2.180042,21.0,8908,30
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395530,2.177060,21.0,8908,27
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394055,2.181299,22.0,8908,27
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393480,2.181555,21.0,8908,21
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391075,2.180223,16.0,8908,39
5,6,"PG. LLUIS COMPANYS, 18 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391349,2.180610,16.0,8908,39
6,7,"PG. PUJADES, 1 (JUTJATS)",ELECTRICBIKESTATION,41.388856,2.183251,16.0,8908,27
7,8,"PG. PUJADES, 2",ELECTRICBIKESTATION,41.389088,2.183568,11.0,8908,27
8,9,"AV. MARQUÉS DE L'ARGENTERA,13",ELECTRICBIKESTATION,41.384532,2.184921,11.0,8908,27
9,11,"PG. MARITIM, 11 (DAVANT PL. BRUGADA)",ELECTRICBIKESTATION,41.381689,2.193914,1.0,8908,21


### Merging datasets

In [20]:
df_merged = pd.merge(estat, info_reduced_3, on='station_id', how='left')

In [21]:
df_merged

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,...,ttl,last_reported_2,last_updated_2,name,physical_configuration,lat,lon,altitude,post_code,capacity
0,1,25,24,1,4,1,1,1,1.569881e+09,True,...,30,2019-09-30 22:00:01,2019-09-30 22:00:20,"C/ GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397952,2.180042,21.0,8908.0,30.0
1,2,12,11,1,15,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:59:14,2019-09-30 22:00:20,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395530,2.177060,21.0,8908.0,27.0
2,3,19,19,0,8,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:59:47,2019-09-30 22:00:20,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394055,2.181299,22.0,8908.0,27.0
3,4,15,15,0,3,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:58:23,2019-09-30 22:00:20,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393480,2.181555,21.0,8908.0,21.0
4,5,35,35,0,3,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:56:25,2019-09-30 22:00:20,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391075,2.180223,16.0,8908.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655818,428,16,16,0,11,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:09,2019-10-31 22:55:30,"C/ PUJADES, 103",ELECTRICBIKESTATION,41.398389,2.196261,11.0,8908.0,27.0
3655819,492,14,14,0,9,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:52,2019-10-31 22:55:30,PL. TETUAN,ELECTRICBIKESTATION,41.394425,2.175103,21.0,1.0,24.0
3655820,494,1,1,0,21,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:01,2019-10-31 22:55:30,RAMBLA CATALUNYA/DIPUTACIO,ELECTRICBIKESTATION,41.389670,2.165428,41.0,8908.0,24.0
3655821,495,0,0,0,24,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:08,2019-10-31 22:55:30,C/ DIPUTACIÓ - TARRAGONA,ELECTRICBIKESTATION,41.376708,2.148395,45.0,8908.0,24.0


In [22]:
df_merged_2 = df_merged[~df_merged.station_id.isin(list_cap)]

df_merged_2

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,...,ttl,last_reported_2,last_updated_2,name,physical_configuration,lat,lon,altitude,post_code,capacity
0,1,25,24,1,4,1,1,1,1.569881e+09,True,...,30,2019-09-30 22:00:01,2019-09-30 22:00:20,"C/ GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397952,2.180042,21.0,8908.0,30.0
1,2,12,11,1,15,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:59:14,2019-09-30 22:00:20,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395530,2.177060,21.0,8908.0,27.0
2,3,19,19,0,8,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:59:47,2019-09-30 22:00:20,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394055,2.181299,22.0,8908.0,27.0
3,4,15,15,0,3,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:58:23,2019-09-30 22:00:20,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393480,2.181555,21.0,8908.0,21.0
4,5,35,35,0,3,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:56:25,2019-09-30 22:00:20,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391075,2.180223,16.0,8908.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655817,427,20,20,0,0,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:54:49,2019-10-31 22:55:30,"C/ DE SANT PAU, 119/ RONDA SANT PAU",ELECTRICBIKESTATION,41.375336,2.168007,24.0,8908.0,20.0
3655819,492,14,14,0,9,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:52,2019-10-31 22:55:30,PL. TETUAN,ELECTRICBIKESTATION,41.394425,2.175103,21.0,1.0,24.0
3655820,494,1,1,0,21,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:01,2019-10-31 22:55:30,RAMBLA CATALUNYA/DIPUTACIO,ELECTRICBIKESTATION,41.389670,2.165428,41.0,8908.0,24.0
3655821,495,0,0,0,24,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:08,2019-10-31 22:55:30,C/ DIPUTACIÓ - TARRAGONA,ELECTRICBIKESTATION,41.376708,2.148395,45.0,8908.0,24.0


In [23]:
# There are some stations in the original dataset "estat" that don't appear in the dataset "informacio".
# I'm going to check which are those stations:

tt = informacio.station_id.to_list()

In [24]:
uu = estat.station_id.to_list()

In [25]:
main_list = np.setdiff1d(uu,tt)

In [26]:
main_list

array([311, 425, 514])

In [27]:
# The stations 311, 425 and 514 don't appear in the dataset "informacio".
# Knowing all this, I'm going to remove the stations that don't appear in the dataset "informacio".

In [28]:
df_merged_3 = df_merged_2[~df_merged_2.station_id.isin(main_list)]

df_merged_3

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,...,ttl,last_reported_2,last_updated_2,name,physical_configuration,lat,lon,altitude,post_code,capacity
0,1,25,24,1,4,1,1,1,1.569881e+09,True,...,30,2019-09-30 22:00:01,2019-09-30 22:00:20,"C/ GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397952,2.180042,21.0,8908.0,30.0
1,2,12,11,1,15,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:59:14,2019-09-30 22:00:20,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395530,2.177060,21.0,8908.0,27.0
2,3,19,19,0,8,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:59:47,2019-09-30 22:00:20,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394055,2.181299,22.0,8908.0,27.0
3,4,15,15,0,3,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:58:23,2019-09-30 22:00:20,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393480,2.181555,21.0,8908.0,21.0
4,5,35,35,0,3,1,1,1,1.569881e+09,True,...,30,2019-09-30 21:56:25,2019-09-30 22:00:20,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391075,2.180223,16.0,8908.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655817,427,20,20,0,0,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:54:49,2019-10-31 22:55:30,"C/ DE SANT PAU, 119/ RONDA SANT PAU",ELECTRICBIKESTATION,41.375336,2.168007,24.0,8908.0,20.0
3655819,492,14,14,0,9,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:52,2019-10-31 22:55:30,PL. TETUAN,ELECTRICBIKESTATION,41.394425,2.175103,21.0,1.0,24.0
3655820,494,1,1,0,21,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:01,2019-10-31 22:55:30,RAMBLA CATALUNYA/DIPUTACIO,ELECTRICBIKESTATION,41.389670,2.165428,41.0,8908.0,24.0
3655821,495,0,0,0,24,1,1,1,1.572562e+09,True,...,30,2019-10-31 22:53:08,2019-10-31 22:55:30,C/ DIPUTACIÓ - TARRAGONA,ELECTRICBIKESTATION,41.376708,2.148395,45.0,8908.0,24.0


In [29]:
df_merged_3.station_id.nunique()

353

### Missing values detection

In [30]:
def missing_percentage(df):
    """This function takes a DataFrame(df) as input and returns two columns, total missing values and total missing values percentage"""
    ## the two following line may seem complicated but its actually very simple. 
    total = df.isnull().sum().sort_values(ascending = False)[df.isnull().sum().sort_values(ascending = False) != 0]
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2)[round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2) != 0]
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])

In [31]:
m = missing_percentage(df_merged_3)
m

,Total,Percent


In [32]:
# There are no missing values in this dataset.

In [33]:
# I'm going to remove the following columns because they are no adding extra information or relevant information:
# last_updated
# last_updated_2
# ttl
# physical_configuration (there is only one type --> ELECTRICBIKESTATION which means there are only stations with
# both electric and mechanic bikes)

df_merged_3.physical_configuration.unique()

array(['ELECTRICBIKESTATION'], dtype=object)

In [34]:
df_merged_4 = df_merged_3.drop(['last_updated', 'last_updated_2', 'ttl', 'physical_configuration'], 1)

df_merged_4

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_reported_2,name,lat,lon,altitude,post_code,capacity
0,1,25,24,1,4,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 22:00:01,"C/ GRAN VIA CORTS CATALANES, 760",41.397952,2.180042,21.0,8908.0,30.0
1,2,12,11,1,15,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 21:59:14,"C/ ROGER DE FLOR, 126",41.395530,2.177060,21.0,8908.0,27.0
2,3,19,19,0,8,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 21:59:47,"C/ NÀPOLS, 82",41.394055,2.181299,22.0,8908.0,27.0
3,4,15,15,0,3,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 21:58:23,"C/ RIBES, 13",41.393480,2.181555,21.0,8908.0,21.0
4,5,35,35,0,3,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 21:56:25,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",41.391075,2.180223,16.0,8908.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655817,427,20,20,0,0,1,1,1,1.572562e+09,True,IN_SERVICE,2019-10-31 22:54:49,"C/ DE SANT PAU, 119/ RONDA SANT PAU",41.375336,2.168007,24.0,8908.0,20.0
3655819,492,14,14,0,9,1,1,1,1.572562e+09,True,IN_SERVICE,2019-10-31 22:53:52,PL. TETUAN,41.394425,2.175103,21.0,1.0,24.0
3655820,494,1,1,0,21,1,1,1,1.572562e+09,True,IN_SERVICE,2019-10-31 22:53:01,RAMBLA CATALUNYA/DIPUTACIO,41.389670,2.165428,41.0,8908.0,24.0
3655821,495,0,0,0,24,1,1,1,1.572562e+09,True,IN_SERVICE,2019-10-31 22:53:08,C/ DIPUTACIÓ - TARRAGONA,41.376708,2.148395,45.0,8908.0,24.0


In [35]:
df_merged_4.status.unique()

array(['IN_SERVICE', 'PLANNED', 'MAINTENANCE', 'NOT_IN_SERVICE'],
      dtype=object)

In [36]:
df_merged_4[df_merged_4['status'] == 'NOT_IN_SERVICE'].station_id.unique()

array([ 36,  84,  12,  69,  13,   6,   4,  72, 346,  31,  22,  15,  18,
       176,  70, 494, 495, 257, 244, 394, 409, 247, 178, 283,  56, 357,
         3, 396, 416, 124, 374, 307, 391, 200,  57, 254, 180, 341,  21,
       296, 100, 127, 188, 350, 303, 211, 426, 359, 387, 316,  59, 360,
       230, 421,   7, 347, 424, 386,  44, 364, 406,   1, 150,  47, 321,
       326, 365, 241,  87, 319, 231, 145,  61, 109,  48, 217, 103, 246,
       412, 105, 300, 354, 216, 301, 141,  79, 400, 402,  26,  78, 186,
       276, 101, 324, 235, 427, 317, 280, 146, 113,  89, 157, 333, 148,
       291, 397, 117, 123, 496,  29, 232, 361, 118])

In [37]:
# All the stations in the dataset df_merged_4 is IN_SERVICE at least at some point during October 2019, because
# 353 is the total amount of stations we have in our current dataset.

In [38]:
df_merged_5 = df_merged_4.copy()

In [39]:
df_merged_5['year'] = [t.year for t in pd.DatetimeIndex(df_merged_5.last_reported_2)]

In [40]:
df_merged_5

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,status,last_reported_2,name,lat,lon,altitude,post_code,capacity,year
0,1,25,24,1,4,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 22:00:01,"C/ GRAN VIA CORTS CATALANES, 760",41.397952,2.180042,21.0,8908.0,30.0,2019
1,2,12,11,1,15,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 21:59:14,"C/ ROGER DE FLOR, 126",41.395530,2.177060,21.0,8908.0,27.0,2019
2,3,19,19,0,8,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 21:59:47,"C/ NÀPOLS, 82",41.394055,2.181299,22.0,8908.0,27.0,2019
3,4,15,15,0,3,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 21:58:23,"C/ RIBES, 13",41.393480,2.181555,21.0,8908.0,21.0,2019
4,5,35,35,0,3,1,1,1,1.569881e+09,True,IN_SERVICE,2019-09-30 21:56:25,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",41.391075,2.180223,16.0,8908.0,39.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655817,427,20,20,0,0,1,1,1,1.572562e+09,True,IN_SERVICE,2019-10-31 22:54:49,"C/ DE SANT PAU, 119/ RONDA SANT PAU",41.375336,2.168007,24.0,8908.0,20.0,2019
3655819,492,14,14,0,9,1,1,1,1.572562e+09,True,IN_SERVICE,2019-10-31 22:53:52,PL. TETUAN,41.394425,2.175103,21.0,1.0,24.0,2019
3655820,494,1,1,0,21,1,1,1,1.572562e+09,True,IN_SERVICE,2019-10-31 22:53:01,RAMBLA CATALUNYA/DIPUTACIO,41.389670,2.165428,41.0,8908.0,24.0,2019
3655821,495,0,0,0,24,1,1,1,1.572562e+09,True,IN_SERVICE,2019-10-31 22:53:08,C/ DIPUTACIÓ - TARRAGONA,41.376708,2.148395,45.0,8908.0,24.0,2019


In [41]:
df_merged_5["month"] = [t.month for t in pd.DatetimeIndex(df_merged_5.last_reported_2)]
df_merged_5["day"] = [t.day for t in pd.DatetimeIndex(df_merged_5.last_reported_2)]
df_merged_5["day_of_week"] = [t.dayofweek for t in pd.DatetimeIndex(df_merged_5.last_reported_2)]
df_merged_5["hour"] = [t.hour for t in pd.DatetimeIndex(df_merged_5.last_reported_2)]
df_merged_5["minute"] = [t.minute for t in pd.DatetimeIndex(df_merged_5.last_reported_2)]
df_merged_5["second"] = [t.second for t in pd.DatetimeIndex(df_merged_5.last_reported_2)]

In [42]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

In [43]:
df_merged_7 = df_merged_5.copy()

In [44]:
month = df_merged_7['month'].values
year = df_merged_7['year'].values
hour = df_merged_7['hour'].values
minute = df_merged_7['minute'].values
num_bikes_av = df_merged_7['num_bikes_available'].values
num_docks_av = df_merged_7['num_docks_available'].values

In [45]:
# Then we extract weekday data by date.weekday() and evenly divide
# a 24-hour day into 72 time slices, such that each time slice
# represents a 20-minute interval.

from datetime import datetime

In [47]:
# Compute IsWeekday
day_of_week = df_merged_7['day_of_week']
IsWeekday = day_of_week < 5
df_merged_7['IsWeekday'] = IsWeekday
# Compute TimeSlice
df_merged_7['TimeSlice'] = (hour*3 + np.floor(minute/20)).astype(int)

In [48]:
df_merged_7.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,...,capacity,year,month,day,day_of_week,hour,minute,second,IsWeekday,TimeSlice
0,1,25,24,1,4,1,1,1,1.569881e+09,True,...,30.0,2019,9,30,0,22,0,1,True,66
1,2,12,11,1,15,1,1,1,1.569881e+09,True,...,27.0,2019,9,30,0,21,59,14,True,65
2,3,19,19,0,8,1,1,1,1.569881e+09,True,...,27.0,2019,9,30,0,21,59,47,True,65
3,4,15,15,0,3,1,1,1,1.569881e+09,True,...,21.0,2019,9,30,0,21,58,23,True,65
4,5,35,35,0,3,1,1,1,1.569881e+09,True,...,39.0,2019,9,30,0,21,56,25,True,65


In [49]:
df_merged_8 = df_merged_7.copy()

In [50]:
df_merged_8.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3147348 entries, 0 to 3655822
Data columns (total 27 columns):
 #   Column                                Dtype         
---  ------                                -----         
 0   station_id                            int64         
 1   num_bikes_available                   int64         
 2   num_bikes_available_types.mechanical  int64         
 3   num_bikes_available_types.ebike       int64         
 4   num_docks_available                   int64         
 5   is_installed                          int64         
 6   is_renting                            int64         
 7   is_returning                          int64         
 8   last_reported                         float64       
 9   is_charging_station                   bool          
 10  status                                object        
 11  last_reported_2                       datetime64[ns]
 12  name                                  object        
 13  lat         

In [51]:
selected_stations = [331, 183, 126, 102]

In [52]:
df_def = df_merged_8[df_merged_8['station_id'].isin(selected_stations)]

In [53]:
df_def

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,...,capacity,year,month,day,day_of_week,hour,minute,second,IsWeekday,TimeSlice
94,102,0,0,0,24,1,1,1,1.569881e+09,True,...,24.0,2019,9,30,0,21,56,55,True,65
118,126,18,18,0,4,1,1,1,1.569881e+09,True,...,23.0,2019,9,30,0,21,58,18,True,65
172,183,28,27,1,0,1,1,1,1.569881e+09,True,...,28.0,2019,9,30,0,21,56,37,True,65
318,331,0,0,0,30,1,1,1,1.569881e+09,True,...,31.0,2019,9,30,0,22,0,5,True,66
504,102,0,0,0,24,1,1,1,1.569881e+09,True,...,24.0,2019,9,30,0,22,1,11,True,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655321,331,0,0,0,31,1,1,1,1.572562e+09,True,...,31.0,2019,10,31,3,22,47,11,True,68
3655507,102,0,0,0,24,1,1,1,1.572562e+09,True,...,24.0,2019,10,31,3,22,52,45,True,68
3655531,126,23,23,0,0,1,1,1,1.572562e+09,True,...,23.0,2019,10,31,3,22,53,50,True,68
3655585,183,21,20,1,7,1,1,1,1.572563e+09,True,...,28.0,2019,10,31,3,22,55,21,True,68


In [54]:
df_def_oct = df_def.copy()

In [55]:
df_def_oct = df_def_oct[df_def_oct['month'] == 10]
df_def_oct

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,...,capacity,year,month,day,day_of_week,hour,minute,second,IsWeekday,TimeSlice
10344,102,0,0,0,24,1,1,1,1.569888e+09,True,...,24.0,2019,10,1,1,0,2,36,True,0
10368,126,17,17,0,5,1,1,1,1.569888e+09,True,...,23.0,2019,10,1,1,0,1,36,True,0
10422,183,28,27,1,0,1,1,1,1.569888e+09,True,...,28.0,2019,10,1,1,0,1,19,True,0
10568,331,0,0,0,30,1,1,1,1.569888e+09,True,...,31.0,2019,10,1,1,0,2,4,True,0
10754,102,0,0,0,24,1,1,1,1.569888e+09,True,...,24.0,2019,10,1,1,0,7,11,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655321,331,0,0,0,31,1,1,1,1.572562e+09,True,...,31.0,2019,10,31,3,22,47,11,True,68
3655507,102,0,0,0,24,1,1,1,1.572562e+09,True,...,24.0,2019,10,31,3,22,52,45,True,68
3655531,126,23,23,0,0,1,1,1,1.572562e+09,True,...,23.0,2019,10,31,3,22,53,50,True,68
3655585,183,21,20,1,7,1,1,1,1.572563e+09,True,...,28.0,2019,10,31,3,22,55,21,True,68


In [56]:
my_dataset = df_def_oct.copy()

In [57]:
df_mapa = my_dataset.drop_duplicates(subset=['station_id'], keep='first')
df_mapa['station_id'] = df_mapa['station_id'].astype(str)
df_mapa

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,is_installed,is_renting,is_returning,last_reported,is_charging_station,...,capacity,year,month,day,day_of_week,hour,minute,second,IsWeekday,TimeSlice
10344,102,0,0,0,24,1,1,1,1.569888e+09,True,...,24.0,2019,10,1,1,0,2,36,True,0
10368,126,17,17,0,5,1,1,1,1.569888e+09,True,...,23.0,2019,10,1,1,0,1,36,True,0
10422,183,28,27,1,0,1,1,1,1.569888e+09,True,...,28.0,2019,10,1,1,0,1,19,True,0
10568,331,0,0,0,30,1,1,1,1.569888e+09,True,...,31.0,2019,10,1,1,0,2,4,True,0


In [58]:
m = folium.Map(zoom_start = 12, location=[41.3887901, 2.1589899])

# Marker clusters
locations = list(zip(df_mapa.lat, df_mapa.lon))
icons = [folium.Icon(icon="bicycle", prefix="fa") for _ in range(len(locations))]

# Create popups
popup_content = []
for station in df_mapa.itertuples():
    number_station = "Station ID: {} ".format(station.station_id)
    content = number_station
    popup_content.append(content)

popups = [folium.Popup(content) for content in popup_content]

cluster = MarkerCluster(locations=locations, icons=icons, popups=popups)
m.add_child(cluster)

m.save("stations_ID_bicing.html")

m

These are the four stations selected for the model building section of this project

In [59]:
# my_dataset.to_csv('ts_dataset.csv')